In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("ibaq_no_cont_or_rev_normalized_against_tot_each_day.csv",header=0,index_col=0)
df

,Day 3,Day 5,Day 10,Day 12,Day 14,Day 18,Day 22,Day 23,Day 26,Day 27,Day 28,Day 30,Day 38,Day 39,Day 40,Day 41,Day 45
sp|P01857|IGHG1_HUMAN,3.465778e-01,0.366554,0.209545,1.413162e-01,0.152028,1.525532e-01,1.533269e-01,0.166128,1.571910e-01,0.167854,1.603228e-01,0.137885,1.601038e-01,1.266374e-01,0.135850,0.152274,1.363352e-01
sp|P12269|IMDH2_CRIGR,3.751970e-04,0.000415,0.000596,6.303580e-04,0.000597,8.048440e-04,7.997850e-04,0.000884,7.703420e-04,0.000834,7.409630e-04,0.000533,6.483040e-04,6.994540e-04,0.000741,0.000732,6.016030e-04
sp|P35950|LDLR_CRIGR;tr|G3HVQ6|G3HVQ6_CRIGR;tr|G3HVQ5|G3HVQ5_CRIGR,7.093080e-05,0.000089,0.000167,1.662340e-04,0.000152,1.534440e-04,1.576730e-04,0.000137,1.826540e-04,0.000170,8.353390e-05,0.000239,2.531140e-04,1.654480e-04,0.000251,0.000263,3.784300e-04
sp|P83337|OFUT1_CRIGR,3.026140e-05,0.000014,0.000011,1.335370e-05,0.000035,4.315140e-05,2.417950e-05,0.000000,3.698220e-05,0.000013,6.127450e-05,0.000037,8.571450e-06,6.887380e-05,0.000025,0.000010,5.598850e-05
tr|A4URF0|A4URF0_CRIGR;tr|G3I474|G3I474_CRIGR,1.080280e-04,0.000131,0.000012,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,6.988610e-06,0.000010,4.515410e-05,0.000018,2.015080e-05,8.501190e-06,0.000032,0.000000,0.000000e+00
tr|C0SSX9|C0SSX9_CRIGR,0.000000e+00,0.000000,0.000000,4.193330e-06,0.000002,7.612450e-07,0.000000e+00,0.000004,0.000000e+00,0.000000,0.000000e+00,0.000000,6.348130e-07,0.000000e+00,0.000000,0.000000,0.000000e+00
tr|F7J0L2|F7J0L2_CRIGR,1.077490e-04,0.000082,0.000549,3.711900e-04,0.000496,2.249290e-04,1.446280e-04,0.000318,1.825660e-04,0.000151,2.415000e-04,0.000269,2.757340e-04,2.616280e-04,0.000266,0.000164,1.038120e-04
tr|G3GR61|G3GR61_CRIGR,0.000000e+00,0.000000,0.000008,1.032130e-05,0.000000,2.088700e-06,2.240540e-05,0.000019,7.554750e-05,0.000003,5.328120e-05,0.000002,2.715980e-05,8.983130e-05,0.000012,0.000007,0.000000e+00
tr|G3GR64|G3GR64_CRIGR,9.464420e-04,0.001010,0.000446,3.056680e-04,0.000342,3.915240e-04,4.006670e-04,0.000431,3.351200e-04,0.000421,4.196250e-04,0.000332,3.650910e-04,3.272980e-04,0.000350,0.000341,4.778030e-04
tr|G3GR69|G3GR69_CRIGR,0.000000e+00,0.000000,0.000000,1.449180e-06,0.000000,3.292340e-06,6.605330e-06,0.000003,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
